In [1]:
##### setup
library(RCurl)
library(jsonlite)


### functions
LOG = function(txt){
    write(txt, file="log.txt")
}

cData.get = function(key){
    return(cData$data[which(names(cData.keys) == key)])
}

top = function(arr, n=5){
    return(sort(arr, decreasing=T)[1:n])
}


### read assets
cData = fromJSON('data/championsData.json')
iData = fromJSON('data/itemsData.json')
## iData.items
iData.items = iData$data
## cData.keys to name
cData.keys = c()
for(champ in cData$data){
    cData.keys[champ$key] = champ$name
}

In [ ]:
### 데이터 불러오기
data = list(length=0)
data.champion = c()
# n = 5
tier = "CHALLENGER/"
for(n in 1:100){
    print(paste("now doing...",n))
    summoner = fromJSON(paste0('data/', tier, n, '/summoner.json'))  # 소환사 이름
    match = fromJSON(paste0('data/', tier, n, '/match.json'))  # 경기시간
    matchList = fromJSON(paste0('data/', tier, n, '/matchList.json')) # 라인/챔피언/경기타입
    participants = fromJSON(paste0('data/', tier, n, '/match_participants_info.json')) # 소환사 명단, index찾으려고
    game = fromJSON(paste0('data/', tier, n, '/match_participants.json')) # 스펠
    stats = fromJSON(paste0('data/', tier, n, '/match_participants_stats.json')) # 게임 정보, 아이템/...
    
    # 소환사 이름
    name = summoner$name
    # 소환사 아이디
    accountId = summoner$accountId
    # 경기시간 (초)
    durations = unlist(match$gameDuration)
    # 라인
    lanes = matchList$lane 
    # 챔피언
    champions = matchList$champion
    # 경기타입, 20=일반, 30,40=랭크
    queues = matchList$queue
    lanes[lanes == "NONE"] = 'BOTTOM?' # 대부분의 NONE은 봇이었음. 이건 쫌 알아봐야.
    # 소환사 index
    index = sapply(participants, function(game) {
        which(sapply(game, function(player) player$accountId) == accountId)
    })
    if(length(index[[1]])==0) next
    # 스펠
    spells = sapply(game, function(g) {
        i = parent.frame()$i
        c(unlist(g$spell1Id[index[i]]), unlist(g$spell2Id[index[i]]))
    })
    # 아이템
    items = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]][3:9]
    })
    # 설치한 와드
    wards.place = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$wardsPlaced
    })
    # 제거한 와드
    wards.kill = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$wardsKilled
    })
    # 와드
    wards = list(placed=wards.place, killed=wards.kill)
    # CS
    cs = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$totalMinionsKilled
    })
    # 승리
    wins = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$win
    })
    # 넣은 데미지
    damages = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$totalDamageDealt
    })
    # 킬
    kills = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$kills
    })
    # 데스
    deaths = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$deaths
    })
    # 어시
    assists = sapply(stats, function(game) {
        i = parent.frame()$i
        game[[index[i]]]$assists
    })

    f1 = which(cData.keys[as.character(champions)] == '신드라')
# #     f2 = which(durations >= 1200) # which의 문제점: 키값을 반환
    filter = f1 # intersect(f1,f2)
    data.champion = c(data.champion, champions)
    data$lane = c(data$lane,lanes[filter])
    data$item = c(data$item, unlist(items[,filter]))
    data$cs = sum(data$cs, cs[filter])
    data$win = sum(data$win, wins[filter])
    data$kill = sum(data$kill, kills[filter])
    data$death = sum(data$death, deaths[filter])
    data$assist = sum(data$assist, assists[filter])
    data$duration = sum(data$duration, durations[filter])
    data$wardKilled = sum(data$wardKilled, wards.kill[filter])
    data$wardPlaced = sum(data$wardPlaced, wards.place[filter])
    data$damage = sum(data$damage, damages[filter])
    data$length = data$length + length(filter)
    
}

In [ ]:
top(table(data.champion), 20)

In [14]:
cData.keys[as.character(names(top(table(data.champion), 20)))]

21             17             22             89            517 
   "미스 포츈"         "티모"         "애쉬"       "레오나"     "사일러스" 
            53             81             51             54             64 
"블리츠크랭크"     "이즈리얼"     "케이틀린"     "말파이트"        "리 신" 
           134            110             99            111            235 
      "신드라"       "바루스"         "럭스"     "노틸러스"         "세나" 
           266             25            222            238            875 
    "아트록스"     "모르가나"       "징크스"         "제드"         "세트"

In [15]:
items_a = table(unlist(sapply(iData.items[as.character(data$item)], function(x) x$name)))
data
# unlist(items[,3:3])

$length
[1] 21

$lane
 [1] "MID"     "MID"     "MID"     "MID"     "MID"     "MID"     "MID"    
 [8] "BOTTOM?" "MID"     "MID"     "BOTTOM?" "TOP"     "MID"     "BOTTOM" 
[15] "BOTTOM"  "JUNGLE"  "BOTTOM?" "BOTTOM"  "JUNGLE"  "BOTTOM?" "TOP"    

$cs
[1] 2294

$win
[1] 13

$kill
[1] 124

$death
[1] 112

$assist
[1] 102

$duration
[1] 33429

$wardKilled
[1] 17

$wardPlaced
[1] 124

$damage
[1] 1867294

$item
                                                                              
 1058  3020  3157  3165  3041  3285  3363  1056  3285  3020  3165  1029     0 
                                                                              
 3364  3020  3285  3157  3165  1082  1058  3340  3020  3165  3285     0     0 
                                                                              
    0  3340  3191  2420  3165  3020  3285  3041  3340  3089     0  3041  3020 
                                                                              
 3285  3165  3340  1056  2420  2003  3285  3165  3020  3340  1056     0  1026 
                                                                              
 3020  3802     0  3340  3285  2421  2003  3165  3191  3020  3340  1056  1052 
                                                                              
 3285  3020  2421  3108  3340  1056     0  1001  3802  1026     0  3340  3191 
                                    item0 item1 item2 item3 item4 item5 item6 
 2420  2003  3020  3165  3285  3340  1056  2031  3157  3020  3285  1058  3363 
                                                                              
 1056  1001  3802  1026     0     0  3364  3853  3020  2420  3165  3191  3285 
                                                                              
 3364  1056  3285  3020  2055  3916     0  3364  1056  3802     0     0     0 
                                                                              
    0  3340  1056  3285     0  3020     0     0  3364  1056  3285  3157  3135 
                                                                              
    0  3020  3364  1056  2003     0     0     0     0  3340  1056  2003  2421 
                        
 1052  3020  3285  3340

In [17]:
lapply(data[3:11], function(x) x/data$length)

$cs
[1] 109.2381

$win
[1] 0.6190476

$kill
[1] 5.904762

$death
[1] 5.333333

$assist
[1] 4.857143

$duration
[1] 1591.857

$wardKilled
[1] 0.8095238

$wardPlaced
[1] 5.904762

$damage
[1] 88918.76

In [18]:
sort(table(unlist(sapply(iData.items[as.character(data$item)], function(x) x$name))), decreasing=T)


       마법사의 신발        루덴의 메아리          도란의 반지 
                  17                   16                   13 
  와드 토템 (장신구)         모렐로노미콘        예언자의 렌즈 
                  13                   10                    6 
           체력 물약        사라진 양피지      존야의 모래시계 
                   5                    4                    4 
              초시계 추적자의 팔목 보호대        망가진 초시계 
                   4                    4                    3 
 메자이의 영혼약탈자        방출의 마법봉   쓸데없이 큰 지팡이 
                   3                    3                    3 
         망원형 개조          속도의 장화          증폭의 고서 
                   2                    2                    2 
       공허의 지팡이    라바돈의 죽음모자            망각의 구 
                   1                    1                    1 
       악마의 마법서          암흑의 인장     얼음 정수의 파편 
                   1                    1                    1 
           제어 와드              천 갑옷          충전형 물약 
                   1                    1                    1 

In [10]:
sapply(iData.items[as.character(data$item)], function(x) x$name)

$`2003`
[1] "체력 물약"

$`3285`
[1] "루덴의 메아리"

$`3916`
[1] "망각의 구"

$`1056`
[1] "도란의 반지"

$`1026`
[1] "방출의 마법봉"

$`3020`
[1] "마법사의 신발"

$`3340`
[1] "와드 토템 (장신구)"

$`3157`
[1] "존야의 모래시계"

$`3285`
[1] "루덴의 메아리"

$`3089`
[1] "라바돈의 죽음모자"

$`3165`
[1] "모렐로노미콘"

$<NA>
NULL

$`3020`
[1] "마법사의 신발"

$`3364`
[1] "예언자의 렌즈"

$`1056`
[1] "도란의 반지"

$`3108`
[1] "악마의 마법서"

$`3067`
[1] "점화석"

$<NA>
NULL

$<NA>
NULL

$`3020`
[1] "마법사의 신발"

$`3340`
[1] "와드 토템 (장신구)"